In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import csv
import optuna

In [4]:
# 셀에서 출력 정확도 설정.
%precision %.3f

'%.3f'

In [5]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

documents = documents[:77] + documents[-77:]

In [6]:
# 전문가 평점 3점 이상인 상위 175개는 label에 1, 나머지 하위는 label에 0 값 부여
# 분류 예측용 label
labels = []
for i in range(154):
    if i < 77:
        labels.append(1)
    else:
        labels.append(0)

In [7]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(documents_corpus, labels, test_size=0.3, random_state=123)

In [9]:
# Tagged Corpus 생성.
# 모델 학습에 필요한 형태로 변환.
my_tagged_corpus = [TaggedDocument(words=my_words, tags=[i]) for i, my_words in enumerate(X_train)]

In [8]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 20, 60)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 10, 30, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Randomforest 하이퍼파라미터 튜닝
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 25)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    # Randomforest 모델 생성
    model = RandomForestClassifier(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features,
                                    random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 13:59:30,990] A new study created in memory with name: no-name-58ac622a-b0d2-4a53-85db-671ec60889ca
[I 2024-06-21 13:59:36,452] Trial 6 finished with value: 0.5641025641025641 and parameters: {'vector_size': 54, 'window': 3, 'min_count': 5, 'epochs': 10, 'negative': 4, 'dm': 0, 'n_estimators': 267, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 6 with value: 0.5641025641025641.
[I 2024-06-21 13:59:37,612] Trial 4 finished with value: 0.5142857142857142 and parameters: {'vector_size': 49, 'window': 9, 'min_count': 1, 'epochs': 10, 'negative': 14, 'dm': 0, 'n_estimators': 111, 'max_depth': 18, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 6 with value: 0.5641025641025641.
[I 2024-06-21 13:59:40,264] Trial 7 finished with value: 0.5 and parameters: {'vector_size': 51, 'window': 2, 'min_count': 9, 'epochs': 20, 'negative': 3, 'dm': 0, 'n_estimators': 276, 'max_depth': 25, 'min_sample

KeyboardInterrupt: 

In [12]:
best_params = {'vector_size': 50, 'window': 7, 'min_count': 10, 'epochs': 20, 'negative': 8, 'dm': 1, 'n_estimators': 369, 'max_depth': 24, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None}

# Doc2Vec 모델 생성
model_rf = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# RandomForest 모델 생성
clf_rf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                            max_depth=best_params['max_depth'],
                            min_samples_split=best_params['min_samples_split'],
                            min_samples_leaf=best_params['min_samples_leaf'],
                            max_features=best_params['max_features'],
                            random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model_rf.infer_vector(doc) for doc in X_train]
X_test_vec = [model_rf.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_rf.fit(X_train_vec, y_train)
y_pred = clf_rf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.69      0.75      0.72        24
           1       0.71      0.65      0.68        23

    accuracy                           0.70        47
   macro avg       0.70      0.70      0.70        47
weighted avg       0.70      0.70      0.70        47



In [10]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 20, 60)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 10, 30, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # SVM 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    gamma = trial.suggest_float('gamma', 1e-4, 1.0, log=True) if kernel in ['rbf', 'poly'] else 'scale'

    # SVM 모델 생성
    model = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        gamma=gamma,
        random_state=123
    )

    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 14:07:21,149] A new study created in memory with name: no-name-edf70c07-5ece-4c3b-81b6-d865cb18626a
[I 2024-06-21 14:07:28,548] Trial 0 finished with value: 0.6571428571428571 and parameters: {'vector_size': 39, 'window': 7, 'min_count': 8, 'epochs': 10, 'negative': 5, 'dm': 1, 'C': 0.02182136984576302, 'kernel': 'poly', 'degree': 3, 'gamma': 0.0007660699344025719}. Best is trial 0 with value: 0.6571428571428571.
[I 2024-06-21 14:07:30,410] Trial 5 finished with value: 0.6571428571428571 and parameters: {'vector_size': 44, 'window': 7, 'min_count': 3, 'epochs': 25, 'negative': 8, 'dm': 0, 'C': 3.4075891069234334, 'kernel': 'poly', 'degree': 3, 'gamma': 0.005221380951221616}. Best is trial 0 with value: 0.6571428571428571.
[I 2024-06-21 14:07:30,731] Trial 6 finished with value: 0.6530612244897959 and parameters: {'vector_size': 29, 'window': 2, 'min_count': 7, 'epochs': 15, 'negative': 6, 'dm': 1, 'C': 0.011688834637356727, 'kernel': 'linear'}. Best is trial 0 with value:

Best hyperparameters:  {'vector_size': 52, 'window': 4, 'min_count': 2, 'epochs': 25, 'negative': 5, 'dm': 1, 'C': 60.91278167538605, 'kernel': 'rbf', 'gamma': 0.06294729784153419}


In [13]:
best_params = {'vector_size': 52, 'window': 4, 'min_count': 2, 'epochs': 25, 'negative': 5, 'dm': 1, 'C': 60.91278167538605, 'kernel': 'rbf', 'gamma': 0.06294729784153419}

# Doc2Vec 모델 생성
model_svc = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# SVM 분류 모델 생성
clf_svc = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'], random_state=123)#degree=best_params['degree'], gamma=best_params['gamma'], 

# Doc2Vec 벡터 생성
X_train_vec = [model_svc.infer_vector(doc) for doc in X_train]
X_test_vec = [model_svc.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_svc.fit(X_train_vec, y_train)
y_pred = clf_svc.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.78      0.88      0.82        24
           1       0.85      0.74      0.79        23

    accuracy                           0.81        47
   macro avg       0.81      0.81      0.81        47
weighted avg       0.81      0.81      0.81        47



In [13]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 20, 60)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 10, 30, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # XGBOOST 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 25)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    gamma = trial.suggest_float('gamma', 0, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    
    # XGboost 모델 생성
    model = XGBClassifier(max_depth=max_depth,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        gamma=gamma,
                        min_child_weight=min_child_weight,
                        random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 14:13:58,935] A new study created in memory with name: no-name-c8923537-1a9c-4448-9007-03524a66b233
[I 2024-06-21 14:14:05,096] Trial 2 finished with value: 0.6222222222222222 and parameters: {'vector_size': 53, 'window': 2, 'min_count': 7, 'epochs': 20, 'negative': 3, 'dm': 0, 'max_depth': 22, 'n_estimators': 229, 'learning_rate': 0.19536068485428004, 'gamma': 0.414484028544014, 'min_child_weight': 4}. Best is trial 2 with value: 0.6222222222222222.
[I 2024-06-21 14:14:07,274] Trial 5 finished with value: 0.6571428571428571 and parameters: {'vector_size': 52, 'window': 7, 'min_count': 2, 'epochs': 20, 'negative': 8, 'dm': 0, 'max_depth': 11, 'n_estimators': 406, 'learning_rate': 0.20349371047960294, 'gamma': 0.8903643118569778, 'min_child_weight': 19}. Best is trial 5 with value: 0.6571428571428571.
[I 2024-06-21 14:14:08,263] Trial 6 finished with value: 0.6571428571428571 and parameters: {'vector_size': 28, 'window': 3, 'min_count': 3, 'epochs': 10, 'negative': 12, 'dm

Best hyperparameters:  {'vector_size': 45, 'window': 2, 'min_count': 4, 'epochs': 15, 'negative': 13, 'dm': 0, 'max_depth': 9, 'n_estimators': 406, 'learning_rate': 0.07515032149234244, 'gamma': 0.1409398514597685, 'min_child_weight': 6}


In [14]:
best_params = {'vector_size': 45, 'window': 2, 'min_count': 4, 'epochs': 15, 'negative': 13, 'dm': 0, 'max_depth': 9, 'n_estimators': 406, 'learning_rate': 0.07515032149234244, 'gamma': 0.1409398514597685, 'min_child_weight': 6}

# Doc2Vec 모델 생성
model_xgb = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# XGBoost 모델 생성
clf_xgb = XGBClassifier(
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    gamma=best_params['gamma'],
    min_child_weight=best_params['min_child_weight'],
    random_state=123
)

# Doc2Vec 벡터 생성
X_train_vec = [model_xgb.infer_vector(doc) for doc in X_train]
X_test_vec = [model_xgb.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_xgb.fit(X_train_vec, y_train)
y_pred = clf_xgb.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.75      0.73        24
           1       0.73      0.70      0.71        23

    accuracy                           0.72        47
   macro avg       0.72      0.72      0.72        47
weighted avg       0.72      0.72      0.72        47



In [16]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 20, 60)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 10, 30, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # LightGBM 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 25)
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 50)

    # LightGBM 모델 생성
    model = LGBMClassifier(
        max_depth=max_depth,
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_samples=min_child_samples,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 14:20:01,573] A new study created in memory with name: no-name-b75d7115-11fa-47f7-bd26-a398ad7e3183
[I 2024-06-21 14:20:08,146] Trial 4 finished with value: 0.7058823529411765 and parameters: {'vector_size': 40, 'window': 7, 'min_count': 8, 'epochs': 15, 'negative': 4, 'dm': 0, 'max_depth': 16, 'num_leaves': 163, 'learning_rate': 0.011247384389874309, 'n_estimators': 384, 'min_child_samples': 48}. Best is trial 4 with value: 0.7058823529411765.
[I 2024-06-21 14:20:09,648] Trial 2 finished with value: 0.7111111111111111 and parameters: {'vector_size': 28, 'window': 4, 'min_count': 1, 'epochs': 15, 'negative': 13, 'dm': 0, 'max_depth': 8, 'num_leaves': 118, 'learning_rate': 0.015616152286161453, 'n_estimators': 97, 'min_child_samples': 8}. Best is trial 2 with value: 0.7111111111111111.
[I 2024-06-21 14:20:10,662] Trial 3 finished with value: 0.6521739130434783 and parameters: {'vector_size': 30, 'window': 9, 'min_count': 5, 'epochs': 20, 'negative': 11, 'dm': 0, 'max_depth

Best hyperparameters:  {'vector_size': 51, 'window': 3, 'min_count': 8, 'epochs': 20, 'negative': 10, 'dm': 1, 'max_depth': 7, 'num_leaves': 104, 'learning_rate': 0.057735942512124536, 'n_estimators': 332, 'min_child_samples': 6}


In [18]:
best_params = {'vector_size': 51, 'window': 3, 'min_count': 8, 'epochs': 20, 'negative': 10, 'dm': 1, 'max_depth': 7, 'num_leaves': 104, 'learning_rate': 0.057735942512124536, 'n_estimators': 332, 'min_child_samples': 6}

# Doc2Vec 모델 생성
model_lgbm = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# LightGBM 모델 생성
clf_lgbm = LGBMClassifier(max_depth=best_params['max_depth'], num_leaves=best_params['num_leaves'], 
                    learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], 
                    min_child_samples=best_params['min_child_samples'], random_state=123)


# Doc2Vec 벡터 생성
X_train_vec = [model_lgbm.infer_vector(doc) for doc in X_train]
X_test_vec = [model_lgbm.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_lgbm.fit(X_train_vec, y_train)
y_pred = clf_lgbm.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

[LightGBM] [Info] Number of positive: 54, number of negative: 53
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1910
[LightGBM] [Info] Number of data points in the train set: 107, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504673 -> initscore=0.018692
[LightGBM] [Info] Start training from score 0.018692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [29]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 20, 60)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 10, 30, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Logistic Regression 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Logistic Regression 모델 생성
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-21 14:25:53,575] A new study created in memory with name: no-name-b553da22-8b1d-48b8-b02d-5113188b66c3
[I 2024-06-21 14:25:56,837] Trial 5 finished with value: 0.56 and parameters: {'vector_size': 48, 'window': 8, 'min_count': 4, 'epochs': 10, 'negative': 3, 'dm': 0, 'C': 0.015941431204116677, 'penalty': 'l2', 'solver': 'saga'}. Best is trial 5 with value: 0.56.
[I 2024-06-21 14:25:58,785] Trial 0 finished with value: 0.6 and parameters: {'vector_size': 22, 'window': 3, 'min_count': 9, 'epochs': 15, 'negative': 13, 'dm': 0, 'C': 0.39996428047858934, 'penalty': 'l2', 'solver': 'liblinear'}. Best is trial 0 with value: 0.6.
[I 2024-06-21 14:25:59,199] Trial 2 finished with value: 0.0 and parameters: {'vector_size': 36, 'window': 8, 'min_count': 5, 'epochs': 10, 'negative': 12, 'dm': 1, 'C': 0.0372164871152174, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 0 with value: 0.6.
c:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning:

Best hyperparameters:  {'vector_size': 34, 'window': 8, 'min_count': 8, 'epochs': 25, 'negative': 10, 'dm': 0, 'C': 0.803915218774452, 'penalty': 'l2', 'solver': 'liblinear'}


In [19]:
best_params = {'vector_size': 34, 'window': 8, 'min_count': 8, 'epochs': 25, 'negative': 10, 'dm': 0, 'C': 0.803915218774452, 'penalty': 'l2', 'solver': 'liblinear'}

# Doc2Vec 모델 생성
model_lr = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=123)

# Logistic Regressor 모델 생성
clf_lr = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'], random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model_lr.infer_vector(doc) for doc in X_train]
X_test_vec = [model_lr.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_lr.fit(X_train_vec, y_train)
y_pred = clf_lr.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.85      0.71      0.77        24
           1       0.74      0.87      0.80        23

    accuracy                           0.79        47
   macro avg       0.80      0.79      0.79        47
weighted avg       0.80      0.79      0.79        47



In [20]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

# 사용하지 않은 2점짜리 자소서 리스트
documents = documents[-175:-77]

In [21]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [24]:
rf_vec = [model_rf.infer_vector(doc) for doc in documents_corpus]
svc_vec = [model_svc.infer_vector(doc) for doc in documents_corpus]
xgb_vec = [model_xgb.infer_vector(doc) for doc in documents_corpus]
lgbm_vec = [model_lgbm.infer_vector(doc) for doc in documents_corpus]
lr_vec = [model_lr.infer_vector(doc) for doc in documents_corpus]

# 총 98개 자소서 예측
print(clf_rf.predict(rf_vec).sum())
print(clf_svc.predict(svc_vec).sum())
print(clf_xgb.predict(xgb_vec).sum())
print(clf_lgbm.predict(lgbm_vec).sum())
print(clf_lr.predict(lr_vec).sum())

38
35
51
42
49


In [23]:
import pickle

with open('d2v_model_rf_small.pkl', 'wb') as f:
    pickle.dump(model_rf, f)

with open('d2v_model_svc_small.pkl', 'wb') as f:
    pickle.dump(model_svc, f)

with open('d2v_model_xgb_small.pkl', 'wb') as f:
    pickle.dump(model_xgb, f)

with open('d2v_model_lgbm_small.pkl', 'wb') as f:
    pickle.dump(model_lgbm, f)

with open('d2v_model_lr_small.pkl', 'wb') as f:
    pickle.dump(model_lr, f)
    
with open('clf_model_rf_small.pkl', 'wb') as f:
    pickle.dump(clf_rf, f)

with open('clf_model_svc_small.pkl', 'wb') as f:
    pickle.dump(clf_svc, f)
    
with open('clf_model_xgb_small.pkl', 'wb') as f:
    pickle.dump(clf_xgb, f)

with open('clf_model_lgbm_small.pkl', 'wb') as f:
    pickle.dump(clf_lgbm, f)

with open('clf_model_lr_small.pkl', 'wb') as f:
    pickle.dump(clf_lr, f)